In [3]:
!pip install geopandas
!pip install geopy

     |████████████████████████████████| 931kB 21.5MB/s eta 0:00:01
     |████████████████████████████████| 1.8MB 33.0MB/s eta 0:00:01
     |████████████████████████████████| 14.7MB 7.0MB/s eta 0:00:011�███████████████████▏     | 12.0MB 22.9MB/s eta 0:00:01
     |████████████████████████████████| 10.9MB 28.2MB/s eta 0:00:01    |████████████▉                   | 4.4MB 28.2MB/s eta 0:00:01     |██████████████████████████████▊ | 10.5MB 28.2MB/s eta 0:00:01
     |████████████████████████████████| 92kB 7.0MB/s eta 0:00:011
  Found existing installation: pyproj 1.9.6
    Uninstalling pyproj-1.9.6:
      Successfully uninstalled pyproj-1.9.6
     |████████████████████████████████| 122kB 5.1MB/s eta 0:00:01


In [18]:
!pip install geocoder

     |████████████████████████████████| 102kB 2.0MB/s ta 0:00:011
     |████████████████████████████████| 829kB 2.9MB/s eta 0:00:01
  Stored in directory: /home/jupyterlab/.cache/pip/wheels/8b/99/a0/81daf51dcd359a9377b110a8a886b3895921802d2fc1b2397e
Successfully built future


In [19]:
import geocoder

In [2]:
import pandas as pd
import numpy as np

In [3]:
df = pd.read_csv('Neighborhood_Data.csv')
df.head()

,Unnamed: 0,Postal Code,Borough,Neighborhood
0,0,M3A,North York,Parkwoods
1,1,M4A,North York,Victoria Village
2,2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [4]:
df.columns

Index(['Unnamed: 0', 'Postal Code', 'Borough', 'Neighborhood'], dtype='object')

In [5]:
df.rename(columns = {'Unnamed: 0': 'index'}, inplace = True)
df = df.set_index('index')
df.head()

,Postal Code,Borough,Neighborhood
index,,,
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [ ]:
Latitude = []
Longitude = []
for index,row in df.iterrows():
    lat_lng_coords = None
    while(lat_lng_coords is None):
      g = geocoder.google('{}, Toronto, Ontario'.format(row["Postal Code"]))
      lat_lng_coords = g.latlng
    Latitude.append(lat_lng_coords[0])
    Longitude.append(lat_lng_coords[1])
df["latitude"] = Latitude
df["longitude"] = Longitude

In [6]:
geo_df = pd.read_csv('Geospatial_Coordinates.csv')
geo_df

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
...,...,...,...
98,M9N,43.706876,-79.518188
99,M9P,43.696319,-79.532242
100,M9R,43.688905,-79.554724
101,M9V,43.739416,-79.588437


In [7]:
df_new = df.merge(geo_df,on = "Postal Code", how = "left")
df_new.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


In [9]:
column_names = ["Postal Code", "Borough", "Neighborhood", "Latitude", "Longitude"]
test_df = pd.DataFrame(columns=column_names)

test_list = ["M5G", "M2H", "M4B", "M1J", "M4G", "M4M", "M1R", "M9V", "M9L", "M5V", "M1B", "M5A"]

for postcode in test_list:
    test_df = test_df.append(df_new[df_new["Postal Code"]==postcode], ignore_index=True)
    
test_df

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
1,M2H,North York,Hillcrest Village,43.803762,-79.363452
2,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
3,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
4,M4G,East York,Leaside,43.709060,-79.363452
5,M4M,East Toronto,Studio District,43.659526,-79.340923
6,M1R,Scarborough,"Wexford, Maryvale",43.750072,-79.295849
7,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest...",43.739416,-79.588437
8,M9L,North York,Humber Summit,43.756303,-79.565963
9,M5V,Downtown Toronto,"CN Tower, King and Spadina, Railway Lands, Har...",43.628947,-79.394420


In [10]:
df_new.to_csv('Neighborhood_final.csv')